In [4]:
from __future__ import annotations

from typing import Tuple, Dict, List, Optional, Union


# Our libraries
import benchmarks # possibile_benchmarks -> {bm4, bm8, bm12, bm15}
import algorithms
import floydwarshall

# Python 3 standard libraries
import random
import copy
import matplotlib.pyplot as plt





def make_picking_list (distances : Dict[int,Dict[int,int]], length : int) -> List[int]:
    """
    Given the distance matrix concerning the whole warehouse and the length of the picking 
    list, this method returns a picking list of randomly selected locations.
    Location 0 is never selected because it is considered to be the I/O point.
    
    :param distances: The distance matrix for the whole warehouse.
    :param length: The length of the picking list.
    :return: The picking list.
    
    """
    candidates = list(range(1, len(distances)))
    picking_list = []
    while len(picking_list) < length:
        location = candidates.pop (random.randint(0, len(candidates) - 1))
        picking_list.append (location)
    return picking_list




def clean_distances (picking_list : List[int], matrix : Dict[int,Dict[int,int]]) -> Dict[int,Dict[int,int]]:
    """
    Given a picking list and a matrix of distances, this method filters the matrix of
    distances keeping only the nodes visited in this specific picking list.
    
    :param picking_list: The picking list.
    :param matrix: The distance matrix.
    :return: A new distance matrix.
    
    """
    m : Dict[int,Dict[int,int]] = {}
    pkl = [0] + list(picking_list)
    
    for node in pkl:
        m[node] = {}
        dic = matrix.get(node)
        
        for node2 in pkl:
            m[node][node2] = dic.get(node2)

    return m



def clean_connections (picking_list : List[int], matrix : Dict[int,Dict[int,Set[int]]]) -> Dict[int,Dict[int,Set[int]]]:
    """
    Given a picking list and a matrix of connections, this method filters the matrix of
    connections keeping only the nodes visited in this specific picking list.
    
    :param picking_list: The picking list.
    :param matrix: The connections matrix.
    :return: A new connections matrix.
    
    """
    m : Dict[int,Dict[int,Set[int]]] = {}
    pkl = [0] + list(picking_list)
    
    for node in pkl:
        m[node] = {}
        dic = matrix.get(node)
        
        for node2 in pkl:
            m[node][node2] = set()
            
            for i in matrix[node][node2]:
                if i in pkl:
                    m[node][node2].add (i)
    return m

In [5]:
wh = floydwarshall.Warehouse (blocks = 3,
                                   racks_per_block = 8,
                                   locations_per_side = 11,
                                   locations_size = (2,2),
                                   aisles_size = 2,
                                   crossaisles_size = 3
                                  )

distances, paths = floydwarshall.floyd_warshall (wh.graph)

picking_list = make_picking_list(distances, 40) 
distance_matrix = clean_distances(picking_list, distances)
connections_matrix = clean_connections(picking_list, paths)

In [6]:
pso_args = {
    'distances': distance_matrix,
    'picking_list': picking_list,
    'paths': connections_matrix,
    'era': 10_000,
    'particles': 40,
    'max_noimp': 900,
    'print_every': 100,
    'finalsearch' : True,
    'particle_data': {
      "beta" : 0.7,
      "check_paths" : 0.15,
      "deepsearch" : 0.0,
      "fulldeepsearch": 0.0,
      "max_depth" : 2500,
      "greediness": 0.1,
    }
}

pso2_args = {
    'distances': distance_matrix,
    'picking_list': picking_list,
    'paths': connections_matrix,
    'era': 10_000,
    'particles': 40,
    'max_noimp': 900,
    'print_every': 100,
    'finalsearch' : False,
    'particle_data': {
      "beta" : 0.7,
      "check_paths" : 0.15,
      "deepsearch" : 0.1,
      "fulldeepsearch": 0.5,
      "max_depth" : 2500,
      "greediness": 0.1,
    }
}

pso3_args = {
    'distances': distance_matrix,
    'picking_list': picking_list,
    'paths': connections_matrix,
    'era': 10_000,
    'particles': 30,
    'max_noimp': 900,
    'print_every': 100,
    'finalsearch' : True,
    'particle_data': {
      "beta" : 0.7,
      "check_paths" : 0.15,
      "deepsearch" : 0.0,
      "fulldeepsearch": 0.0,
      "max_depth" : 2500,
      "greediness": 0.1,
    }
}

pso4_args = {
    'distances': distance_matrix,
    'picking_list': picking_list,
    'paths': connections_matrix,
    'era': 10_000,
    'particles': 30,
    'max_noimp': 900,
    'print_every': 100,
    'finalsearch' : False,
    'particle_data': {
      "beta" : 0.7,
      "check_paths" : 0.15,
      "deepsearch" : 0.1,
      "fulldeepsearch": 0.5,
      "max_depth" : 2500,
      "greediness": 0.1,
    }
}

In [7]:
pso = algorithms.Mattia_PSO(**pso_args)
pso2 = algorithms.Mattia_PSO(**pso2_args)
pso3 = algorithms.Mattia_PSO(**pso3_args)
pso4 = algorithms.Mattia_PSO(**pso4_args)
zhong = algorithms.Zhong_PSO (distances = distance_matrix,
                                      picking_list=picking_list,
                                      particles = 30,
                                      w = 0.8,
                                      lt = 1000,
                                      max_iter = 10_000,
                                      max_noimp = 900,
                                      print_every = 100
                             )

In [8]:
from algorithms.utils import Race

In [10]:
race = Race([pso])
race(n=5)
for res in race.results:
    print(res["cost"], res["computations"], res["time"])

Mattia_PSO [1/5]Mattia_PSO [2/5]

Mattia_PSO [3/5]Mattia_PSO [4/5]
Mattia_PSO [5/5]

592 5960 27.175536394119263
596 86111 28.35066533088684
608 75833 28.2583224773407
592 185204 33.43187403678894
596 88769 21.610538959503174


In [11]:
race = Race([pso2])
race(n=5)
for res in race.results:
    print(res["cost"], res["computations"], res["time"])

Mattia_PSO [1/5]
Mattia_PSO [2/5]
Mattia_PSO [3/5]Mattia_PSO [4/5]
Mattia_PSO [5/5]

580 6292114 453.10235023498535
580 4795719 348.40981435775757
580 712160 299.09872579574585
580 755378 247.13452911376953
580 6064547 361.99502825737


In [12]:
race = Race([pso3])
race(n=5)
for res in race.results:
    print(res["cost"], res["computations"], res["time"])

Mattia_PSO [1/5]Mattia_PSO [2/5]

Mattia_PSO [3/5]Mattia_PSO [4/5]

Mattia_PSO [5/5]
628 96933 16.141084909439087
596 51111 28.25708842277527
584 71322 32.52517223358154
580 54075 19.18925905227661
604 197163 22.02628207206726


In [13]:
race = Race([pso4])
race(n=5)
for res in race.results:
    print(res["cost"], res["computations"], res["time"])

Mattia_PSO [2/5]Mattia_PSO [1/5]Mattia_PSO [3/5]
Mattia_PSO [4/5]


Mattia_PSO [5/5]
580 2234460 263.01098895072937
580 5122251 258.56687784194946
580 383667 180.80460119247437
580 2024607 257.4478394985199
580 4416279 302.02969670295715


In [14]:
race = Race([zhong])
race(n=5)
for res in race.results:
    print(res["cost"], res["computations"], res["time"])

Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
580 71892 58.070708990097046
580 59433 59.76802349090576
580 97161 67.07068467140198
580 75813 34.13342046737671
580 70173 52.096745014190674
